In [1]:
import pandas as pd
import xmltodict
import requests
import os
import numpy as np

In [2]:
#import ddf_utils.ddf_reader as ddf
from ddf_utils.str import to_concept_id

# get all meta data

In [3]:
api = 'http://apps.who.int/gho/athena/api/'

In [4]:
xml = requests.get(api)

In [5]:
xml = xml.content

In [6]:
p = xmltodict.parse(xml)

In [7]:
p['GHO']['Metadata']['Attribute'][0]

OrderedDict([('@Label', 'COMMENT'),
             ('@EntityType', 'CORE_DIMENSION'),
             ('@Entity', 'DATASOURCE'),
             ('Display', 'COMMENT')])

# get all indicators

In [8]:
# import asyncio

In [8]:
inds = 'http://apps.who.int/gho/athena/api/GHO/'
xml = requests.get(inds)
xml = xml.content

In [9]:
indi = xmltodict.parse(xml)

In [10]:
indi_list = []

for i in indi['GHO']['Metadata']['Dimension']['Code']:
#     print(i)
    indi_list.append(i['@Label'])

In [12]:
len(indi_list)

2796

In [14]:
!open ../

In [13]:
failed = []
successed = []

In [16]:
def download(i):
    url = 'http://apps.who.int/gho/athena/data/data-coded.csv?target=GHO/{}'.format(i)
    try:
        df = pd.read_csv(url)
        df.to_csv('../source/{}.csv'.format(i))
    except:
        failed.append(i)
    successed.append(i)

In [ ]:
# pool_size = 5

In [14]:
# @asyncio.coroutine
# def run_download():
#     todos = [x for x in indi_list if x not in successed]
#     s = 0
#     while s + pool_size < len(todos): 
#         for t in todos[s:s+pool_size]:
#             yield from download(t)
#         s = s + pool_size
#     for t in todos[s:]:
#             yield from download(t)

In [17]:
indi['GHO']['Metadata']['Dimension']['Code'][33]

OrderedDict([('@Label', 'GBD_0000000038'),
             ('@DisplaySequence', '0'),
             ('@URL',
              'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=3367'),
             ('Attr',
              [OrderedDict([('@Category', 'DEFINITION_XML'),
                            ('Value',
                             OrderedDict([('Display',
                                           'http://apps.who.int/gho/indicatorregistryservice/publicapiservice.asmx/IndicatorGetAsXml?profileCode=WHO&applicationCode=System&languageAlpha2=en&indicatorId=3367')]))]),
               OrderedDict([('@Category', 'CATEGORY'),
                            ('Value',
                             OrderedDict([('Display',
                                           'Mortality and global health estimates')]))])]),
             ('Display', 'GBD - crude death rate')])

In [18]:
indi_desc_list = []
links = []

for i in indi['GHO']['Metadata']['Dimension']['Code']:
    indi_desc_list.append(i['Display'])
    if '@URL' in i:
        links.append(i['@URL'])
    else:
        links.append(None)

In [20]:
indi_desc_list[:10]

['Infant mortality rate (probability of dying between birth and age 1 per 1000 live births)',
 'Adolescent birth rate (per 1000 women aged 15-19 years)',
 'Contraceptive prevalence (%)',
 'Unmet need for family planning (%)',
 'Under-five mortality rate (probability of dying by age 5 per 1000 live births)',
 'Median availability of selected generic medicines (%) - Public',
 'Median consumer price ratio of selected generic medicines - Public',
 'Antiretroviral therapy coverage among people with advanced HIV infection (%)',
 'Children aged <5 years sleeping under insecticide-treated nets (%)',
 'Children aged <5 years with fever who received treatment with any antimalarial (%)']

In [21]:
links[:10]

['http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=1',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=4669',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=5',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=6',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=7',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=10',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=11',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=12',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=13',
 'http://apps.who.int/gho/indicatorregistry/App_Main/view_indicator.aspx?iid=14']

In [15]:
conc = pd.DataFrame([], columns=['concept', 'concept_type', 'description'])

In [16]:
conc['concept'] = indi_list
conc['description'] = indi_desc_list

In [14]:
conc['concept_type'] = 'measure'

In [31]:
conc.to_csv('../../ddf--concepts--continuous.csv', index=False)

In [122]:
!open .

In [88]:
# inds = pd.read_csv('all_indicators.csv')

In [22]:
conc_ = pd.DataFrame([], columns=['concept', 'description', 'size', 'link'])
conc_['concept'] = indi_list
conc_['description'] = indi_desc_list
conc_['link'] = links

In [26]:
lens = []

for i in conc_['concept'].values:
    path = os.path.join('../source/', i+'.csv')
    if os.path.exists(path):
        try:
            df = pd.read_csv(path)
            lens.append(len(df))
        except:
            print(path)
            lens.append(-1)
    else:
        lens.append(np.nan)

../source/ebreast5.csv
../source/itnpreg.csv
../source/pncall5.csv
../source/pnchome5.csv
../source/sba5.csv
../source/uweight3.csv
../source/tfr.csv
../source/u5mr.csv
../source/SA_0000001699.csv
../source/SA_0000001701.csv
../source/SA_0000001708.csv
../source/SA_0000001715.csv
../source/MEDS5070100.csv
../source/PercposSW.csv
../source/PercposMSM.csv
../source/HIV_0000000026.csv
../source/RS_239.csv
../source/RS_240.csv
../source/RS_241.csv
../source/WHS_PBR.csv
../source/MEDS1_01_01.csv
../source/MEDS1_01_03.csv
../source/MEDS1_01_05.csv
../source/MEDS1_02_12.csv
../source/MEDS1_02_13.csv
../source/MEDS1_02_16.csv
../source/MEDS8_02_14.csv
../source/CCO_1.csv
../source/MEDS2_01_15_02_01.csv
../source/MEDS8_01_12_01.csv
../source/M_Est_cig_daily.csv
../source/M_survey_title_adult.csv
../source/M_svy_age_adult.csv
../source/M_def_1_prev_adult.csv
../source/IR_INSECTICIDERESISTANCE_PREV.csv
../source/IR_ORGANOCHLORINE_PRESENCE.csv
../source/IR_ORGANOPHOSPHATE_PRESENCE.csv
../source/IR

/Users/semio/.pyenv/versions/3.6.5/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
conc_['size'] = lens

In [28]:
conc_.to_csv('../all_indicators.csv', index=False)

In [32]:
dtype = []
key = []

for i in conc['concept'].values:
    path = os.path.join('source', i+'.csv')
    if os.path.exists(path):
        try:
            df = pd.read_csv(path)
        except:
            dtype.append(np.nan)
            key.append(np.nan)
            continue
        
        if len(df) == 0:
            dtype.append(np.nan)
            key.append(np.nan)
            continue
            
        if np.all(df['Numeric'].isnull()) == True:
            dtype.append('str')
        else:
            dtype.append('number')
            
        if 'COUNTRY' in df.columns:
            key.append('country')
        elif 'REGION' in df.columns:
            key.append('region')
        else:
            key.append('unknow')
        
    else:
        dtype.append(np.nan)
        key.append(np.nan)

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
conc['length'] = lens

In [34]:
conc['data_key'] = key
conc['data_type'] = dtype

In [36]:
conc[['concept', 'concept_type', 'length', 'data_key', 'data_type', 'description']].to_csv('all_indicators.csv', index=False)

In [38]:
conc[conc['length'] == -1]['concept'].values

array(['HRH_20', 'HRH_27', 'DEVICES12', 'DEVICES21', 'DEVICES22',
       'WHS9_CBR', 'PerctestedANC', 'PercposANC', 'PercposSW',
       'PercposMSM', 'MORT_700', 'GHE_DALYNUM', 'GHE_YLDNUM',
       'GHE_YLDRATE', 'GHE_YLLNUM', 'GHE_YLLRATE', 'PerctreatedANC',
       'csection', 'EMFLIMITPOWERDENSITY'], dtype=object)

In [118]:
inds['data_type'].unique()

array(['number', nan], dtype=object)

In [105]:
k = np.random.choice(inds['concept'].values)
print(k)

Camp_mat_tested


In [74]:
k = 'MDG_0000000018'

In [75]:
conc[conc['concept'] == k]['description']

13    Deaths due to tuberculosis among HIV-positive ...
Name: description, dtype: object

In [77]:
df = pd.read_csv('source/{}.csv'.format(k))
df.sort_values(by=['REGION', 'YEAR'])

,GHO,PUBLISHSTATE,YEAR,REGION,WORLDBANKINCOMEGROUP,Display Value,Numeric,Low,High,Comments
10,MDG_0000000018,PUBLISHED,2007,AFR,NaN,47.6,47.6,47.6,47.6,NaN
9,MDG_0000000018,PUBLISHED,2007,AMR,NaN,0.9,0.9,0.9,0.9,NaN
4,MDG_0000000018,PUBLISHED,2007,EMR,NaN,1.4,1.4,1.4,1.4,NaN
8,MDG_0000000018,PUBLISHED,2007,EUR,NaN,0.9,0.9,0.9,0.9,NaN
0,MDG_0000000018,PUBLISHED,2007,GLOBAL,NaN,6.8,6.8,6.8,6.8,NaN
1,MDG_0000000018,PUBLISHED,2007,NOTSPEC,WB_HI,0.1,0.1,0.1,0.1,NaN
2,MDG_0000000018,PUBLISHED,2007,NOTSPEC,WB_LI,21.1,21.1,21.1,21.1,NaN
3,MDG_0000000018,PUBLISHED,2007,NOTSPEC,WB_WORLD_INCOME,6.8,6.8,6.8,6.8,NaN
6,MDG_0000000018,PUBLISHED,2007,NOTSPEC,WB_UMI,12.9,12.9,12.9,12.9,NaN
11,MDG_0000000018,PUBLISHED,2007,NOTSPEC,WB_LMI,2.1,2.1,2.1,2.1,NaN


In [ ]:
ddf.SEARCH_PATH = '/Users/semio/src/work/Gapminder/'

In [73]:
ddf.ddf_datapoint('ddf--gapminder--systema_globalis', 'tb_with_hivplus_number_of_new_cases_estimated').query('geo== "ago"')

,geo,tb_with_hivplus_number_of_new_cases_estimated,time
72,ago,595,1990
73,ago,772,1991
74,ago,992,1992
75,ago,1262,1993
76,ago,1585,1994
77,ago,1963,1995
78,ago,2399,1996
79,ago,2890,1997
80,ago,3434,1998
81,ago,4030,1999


In [108]:
df['Display Value'].unique()

array(['Not applicable', 'Not available', 'No', 'Yes'], dtype=object)

In [119]:
np.all(df['Numeric'].isnull()) == True

False

In [120]:
df['Numeric'].unique() # 所有indicator都是有numeric数据的。那display Value怎么储存？

array([  18.,   56.,   44.,   50.,   62.,   24.,   87.,   91.,   75.,
         28.,   29.,   58.,   73.,   78.,   48.,   69.,   57.,   33.,
         27.,   26.,   63.,   55.,   68.,   40.,   66.,   79.,   67.,
         46.,   30.,   64.,   43.,   74.,  100.,   92.,   83.,   88.,
         61.,   71.,   99.,   90.,   86.,   85.,   81.,   60.,   97.,
         70.,   89.,   93.,   72.,   84.,   35.,   95.,   45.,   96.,
         76.,    6.,   65.,   82.,   94.,   77.,   80.,   59.,   98.,
         47.,   52.,   51.,   42.,   nan])

In [85]:
conc.columns

Index(['concept', 'concept_type', 'description', 'length', 'data_key',
       'data_type'],
      dtype='object')

In [78]:
# data points

In [131]:
conc = pd.read_csv('all_indicators.csv')

In [132]:
for i, row in conc[(conc['length'] > 0) & (conc['data_key'] == 'country')].iterrows():
    cid =  to_concept_id(row['concept'])
    df = pd.read_csv(os.path.join('source', row['concept']+'.csv'))   
    df.columns = list(map(to_concept_id, df.columns))    
    path = os.path.join('../', 'ddf--datapoints--{}--by--country--year.csv'.format(cid))
    
    if 'year' not in df.columns:
#         print(cid)
        continue
    
    if row['data_type'] == 'number':
        df = df[['country', 'year', 'numeric']]
    else:
        df = df[['country', 'year', 'display_value']]
        
    df.columns = [['country', 'year', cid]]
    
    df.sort_values(by=['country', 'year']).dropna(how='any').to_csv(path, index=False)
    

In [89]:
conc['concept'] = conc['concept'].map(to_concept_id)

In [ ]:
conc.query()

In [90]:
# country

In [91]:
c_url = 'http://apps.who.int/gho/athena/api/COUNTRY'

cr = requests.get(c_url)

In [93]:
cp = xmltodict.parse(cr.content)

In [112]:
c = {}

for i in cp['GHO']['Metadata']['Dimension']['Code']:
    
    attrs = i['Attr']
    
    c[i['@Label']] = dict([(attr['@Category'], attr['Value']['Display']) for attr in attrs])

In [113]:
cdf = pd.DataFrame.from_dict(c, orient='index')

In [116]:
cdf.index.name = 'country'

cdf.reset_index().to_csv('all_countries.csv', index=False)

In [117]:
cdf = cdf.reset_index()

In [118]:
!open .

In [152]:
cdf[['country', 'SHORTNAMEES']].rename(columns={'SHORTNAMEES': 'name'}).to_csv('../ddf--entities--country.csv', index=False)

In [121]:
# concepts

In [123]:
not_conc = ["m_survey_title_adult",
"m_svy_national_adult",
"m_svy_age_adult",
"m_def_1_text_adult",
"m_def_1_prev_adult",
"m_def_2_text_adult",
"m_def_2_prev_adult",
"m_smkless_survey_title_adult",
"m_smkless_svy_national_adult",
"m_smkless_age_range_adult",
"m_smkless_def_text_adult",
"m_smkless_prev_adult",
"m_survey_title_youth",
"m_svy_national_youth",
"m_svy_age_youth",
"m_def_1_text_youth",
"m_def_1_prev_youth",
"m_def_2_text_youth",
"m_def_2_prev_youth",
"m_smkless_survey_title_youth",
"m_smkless_svy_national_youth",
"m_smkless_age_range_youth",
"m_smkless_def_text_youth",
"m_smkless_prev_youth",
"camp_nat",
"camp_gov_prog",
"camp_mat_tested",
"camp_aud_research",
"camp_tv_radio",
"camp_media_planning",
"camp_news",
"camp_airtime",
"camp_eval_impact",
"rev_type",
"rev_year",
"rev_curr",
"rev_excise",
"rev_vat",
"rev_imp_other",
"rev_govt_total",
"r_excise_type",
"r_excise_uniform_varied",
"r_excise_specific_reliance",
"r_excise_min_specific",
"r_excise_retail_price_base",
"r_afford_less",
"r_afford_tax_auto_adjust",
"r_admin_tax_stamps",
"r_admin_duty_free_limited",
"r_admin_duty_free_allowance",
"r_excise_proportion",
"r_afford_gdp",
"r_afford_price_dispersion"]

In [136]:
conc_ = conc.copy()

conc_['concept'] = conc_['concept'].map(to_concept_id)

In [137]:
conc_ = conc_.query('concept not in {}'.format(not_conc))

In [138]:
conc_ = conc_.query('length > 0')

In [145]:
condf = conc_[['concept', 'concept_type', 'description']]

In [146]:
condf = condf.rename(columns={'description': 'name'})

In [147]:
condf = condf.append(pd.DataFrame([['name', 'string', 'Name'], 
                                   ['year', 'time', 'Year'], 
                                   ['country', 'entity_domain', 'Country']], columns=condf.columns))

In [149]:
condf.to_csv('../ddf--concepts.csv', index=False)

In [150]:
from ddf_utils.index import create_index_file

In [153]:
create_index_file('../')

,key,value,file
0,concept,concept_type,ddf--concepts.csv
1,concept,name,ddf--concepts.csv
0,"country,year",air_1,ddf--datapoints--air_1--by--country--year.csv
0,"country,year",air_10,ddf--datapoints--air_10--by--country--year.csv
0,"country,year",air_11,ddf--datapoints--air_11--by--country--year.csv
0,"country,year",air_12,ddf--datapoints--air_12--by--country--year.csv
0,"country,year",air_13,ddf--datapoints--air_13--by--country--year.csv
0,"country,year",air_14,ddf--datapoints--air_14--by--country--year.csv
0,"country,year",air_15,ddf--datapoints--air_15--by--country--year.csv
0,"country,year",air_16,ddf--datapoints--air_16--by--country--year.csv


## script

In [170]:
import pycurl

In [171]:
air6 = 'http://apps.who.int/gho/athena/api/GHO/{}.csv'.format('MORT_100')

In [173]:
air6

'http://apps.who.int/gho/athena/api/GHO/MORT_100.csv'

In [ ]:
with open('test.csv', 'wb') as f:
    c = pycurl.Curl()
    c.setopt(c.URL, air6)
    c.setopt(c.FOLLOWLOCATION, True)
    c.setopt(c.WRITEDATA, f)
    c.perform()
    c.close()

In [167]:
r = requests.get(air6)

In [161]:
with open('source/AIR_6.csv', 'w') as f:
    f.write(r.text)

In [169]:
r.text

'<html>\n<head>\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n  <meta http-equiv="refresh" content="0;url=en/">\n</head>\n</html>\n'

In [ ]:
requests.get?